# Getting Started with LangChain for AI Applications in Python

Welcome to the LangChain Getting Started Guide! This guide will help you understand the basics of LangChain and how to use it to build AI applications in Python.

If you haven't already, please check out the associated blog post for this tutorial: [Getting Started with LangChain for AI Applications in Python](https://www.ubermensch.dev/getting-started-with-langchain-for-ai-applications-in-python)

You can also find more guides and tutorials on the [Ubermensch Developer](https://www.ubermensch.dev) website.


# Imports

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-core openai faiss-cpu python-dotenv

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains import SimpleSequentialChain
import asyncio
from typing import List

# Note: You'll need to set up your OpenAI API key in a .env file
# Copy .env.example to .env and add your API key
from dotenv import load_dotenv
load_dotenv()

True

# Initialize the LLM

In [26]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o"
)

# Simple First Workflow

## Single-Step Chain

In [23]:
prompt = PromptTemplate(template="Answer this: {question}", input_variables=["question"])

chain = LLMChain(llm=llm, prompt=prompt)

print(chain.invoke("What is LangChain?"))

{'question': 'What is LangChain?', 'text': 'LangChain is a framework designed to facilitate the development of applications powered by large language models (LLMs). It provides tools and abstractions that help developers build complex applications by leveraging the capabilities of LLMs. LangChain is particularly useful for creating applications that involve tasks such as natural language understanding, text generation, and conversational interfaces.\n\nThe framework offers several key features, including:\n\n1. **Prompt Management**: Tools for managing and optimizing prompts to improve the performance and accuracy of language models.\n\n2. **Chaining**: The ability to chain together multiple LLM calls or combine them with other computational steps to create more sophisticated workflows.\n\n3. **Memory**: Mechanisms to maintain state or context across interactions, which is crucial for applications like chatbots or interactive agents.\n\n4. **Integration**: Support for integrating with 

## Multi-Step Chain

In [24]:
question_prompt = PromptTemplate(
    template="Generate a specific question about the Python framework LangChain's {aspect}",
    input_variables=["aspect"]
)
question_chain = LLMChain(llm=llm, prompt=question_prompt)

answer_prompt = PromptTemplate(
    template="Answer this question in detail: {question}",
    input_variables=["question"]
)
answer_chain = LLMChain(llm=llm, prompt=answer_prompt)

sequential_chain = SimpleSequentialChain(
    chains=[question_chain, answer_chain],
    verbose=True
)

print(sequential_chain.invoke("architecture"))



> Entering new SimpleSequentialChain chain...
How does LangChain's architecture facilitate the integration of large language models with external data sources and tools to enhance natural language processing capabilities?
LangChain is a framework designed to facilitate the integration of large language models (LLMs) with external data sources and tools, thereby enhancing natural language processing (NLP) capabilities. Its architecture is modular and flexible, allowing developers to build complex applications that leverage the power of LLMs in conjunction with various data sources and computational tools. Here’s a detailed look at how LangChain achieves this:

### 1. Modular Architecture

LangChain's architecture is inherently modular, which means it is composed of interchangeable components that can be easily integrated or replaced. This modularity allows developers to customize and extend the framework to suit specific use cases. The key components include:

- **Language Models**: A

# Exploring Some Advanced LangChain Features

## Async Execution with Batching

In [29]:
multiply_prompt = PromptTemplate(
    template="What is {number} multiplied by itself (i.e., {number} x {number})? Just respond with the number, no explanation.",
    input_variables=["number"]
)

verify_prompt = PromptTemplate(
    template="The LLM calculated that {number} x {number} equals {llm_result}. The actual result is {actual_result}. Did the LLM calculate correctly? Answer yes or no.",
    input_variables=["number", "llm_result", "actual_result"]
)

multiply_chain = LLMChain(llm=llm, prompt=multiply_prompt)
verify_chain = LLMChain(llm=llm, prompt=verify_prompt)

async def process_number(number: int, batch_num: int):
    actual_result = number * number
    
    llm_result = await multiply_chain.ainvoke({"number": number})
    
    verification = await verify_chain.ainvoke({
        "number": number,
        "llm_result": llm_result["text"],
        "actual_result": actual_result
    })
    
    return {
        "number": number,
        "batch": batch_num,
        "llm_result": llm_result["text"],
        "actual_result": actual_result,
        "verification": verification["text"]
    }

def chunk_list(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

async def main():
    numbers_list = list(range(1, 11))
    batch_size = 3
    
    batches = chunk_list(numbers_list, batch_size)
    tasks = []
    
    for batch_num, batch in enumerate(batches, 1):
        batch_tasks = [process_number(num, batch_num) for num in batch]
        tasks.extend(batch_tasks)
    
    results = await asyncio.gather(*tasks)
    return results

results = await main()

for result in results:
    print(f"\nBatch {result['batch']} - Number: {result['number']}")
    print(f"LLM's result: {result['llm_result']}")
    print(f"Actual result: {result['actual_result']}")
    print(f"Verification: {result['verification']}")


Batch 1 - Number: 1
LLM's result: 1
Actual result: 1
Verification: Yes.

Batch 1 - Number: 2
LLM's result: 4
Actual result: 4
Verification: Yes.

Batch 1 - Number: 3
LLM's result: 9
Actual result: 9
Verification: Yes.

Batch 2 - Number: 4
LLM's result: 16
Actual result: 16
Verification: Yes.

Batch 2 - Number: 5
LLM's result: 25
Actual result: 25
Verification: Yes.

Batch 2 - Number: 6
LLM's result: 36
Actual result: 36
Verification: Yes.

Batch 3 - Number: 7
LLM's result: 49
Actual result: 49
Verification: Yes.

Batch 3 - Number: 8
LLM's result: 64
Actual result: 64
Verification: Yes.

Batch 3 - Number: 9
LLM's result: 81
Actual result: 81
Verification: Yes.

Batch 4 - Number: 10
LLM's result: 100
Actual result: 100
Verification: Yes.


## Adding Memory for Contextual Conversations

In [9]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

# Initialize the memory
memory = ConversationBufferMemory(return_messages=True)

# Initialize the conversation chain
math_tutor = ConversationChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o-mini"),
    memory=memory,
    verbose=True, 
    prompt=PromptTemplate(
        input_variables=["history", "input"],
        template="""You are a helpful math tutor.

Previous conversation:
{history}

Student: {input}"""
    )
)

async def demonstrate_math_tutor():
    responses = []
    
    print("\n=== Question 1: Solving x² = 16 ===")
    print("Student: Can you help me solve x² = 16?")
    response = await math_tutor.arun("Can you help me solve x² = 16?")
    print("\nTutor:", response)
    responses.append(response)
    
    print("\n=== Question 2: Changing to x² = 25 ===")
    print("Student: What if we changed the right side to 25? Would it be harder?")
    response = await math_tutor.arun("What if we changed the right side to 25? Would it be harder?")
    print("\nTutor:", response)
    responses.append(response)
    
    print("\n=== Question 3: Pattern Recognition ===")
    print("Student: Do you notice any pattern between these two problems we solved?")
    response = await math_tutor.arun("Do you notice any pattern between these two problems we solved?")
    print("\nTutor:", response)
    responses.append(response)
    
    print("\n=== Question 4: Challenge Problem ===")
    print("Student: Can you give me a slightly harder problem that builds on what we've learned?")
    response = await math_tutor.arun("Can you give me a slightly harder problem that builds on what we've learned?")
    print("\nTutor:", response)
    responses.append(response)
    
    return responses

responses = await demonstrate_math_tutor()

print("\n=== Full Conversation History ===")
for i, response in enumerate(responses, 1):
    print(f"\nInteraction {i}:")
    print(f"Tutor: {response}")

print("\n=== Memory Contents ===")
for message in memory.chat_memory.messages:
    print(f"{message.type}: {message.content}")


=== Question 1: Solving x² = 16 ===
Student: Can you help me solve x² = 16?


> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful math tutor.

Previous conversation:
[]

Student: Can you help me solve x² = 16?

> Finished chain.

Tutor: Of course! To solve the equation \( x^2 = 16 \), you want to find the values of \( x \) that make this equation true.

1. Start by taking the square root of both sides of the equation. Remember that when you take the square root, you need to consider both the positive and negative roots:

   \[
   x = \pm \sqrt{16}
   \]

2. Calculate the square root of 16:

   \[
   \sqrt{16} = 4
   \]

3. So, the solutions are:

   \[
   x = 4 \quad \text{and} \quad x = -4
   \]

Therefore, the solutions to the equation \( x^2 = 16 \) are \( x = 4 \) and \( x = -4 \).

=== Question 2: Changing to x² = 25 ===
Student: What if we changed the right side to 25? Would it be harder?


> Entering new ConversationChain chain...
Prompt after f

## Custom Runnables

In [11]:
from langchain_core.runnables import RunnableLambda

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

prompt = PromptTemplate(
    template="Answer this: {question}",
    input_variables=["question"]
)
base_chain = LLMChain(llm=llm, prompt=prompt)

def custom_uppercase(msg):
    return {"output": msg["text"].upper()}

custom_chain = base_chain | RunnableLambda(custom_uppercase)

result = custom_chain.invoke({"question": "What is LangChain?"})
print(result)

{'output': 'LANGCHAIN IS A FRAMEWORK DESIGNED TO FACILITATE THE DEVELOPMENT OF APPLICATIONS THAT UTILIZE LARGE LANGUAGE MODELS (LLMS). IT PROVIDES A SET OF TOOLS AND COMPONENTS THAT HELP DEVELOPERS BUILD APPLICATIONS THAT CAN INTERACT WITH LLMS IN A MORE STRUCTURED AND EFFICIENT WAY. LANGCHAIN SUPPORTS VARIOUS FUNCTIONALITIES, INCLUDING:\n\n1. **PROMPT MANAGEMENT**: IT ALLOWS DEVELOPERS TO CREATE, MANAGE, AND OPTIMIZE PROMPTS FOR LLMS TO IMPROVE THE QUALITY OF RESPONSES.\n\n2. **CHAINS**: LANGCHAIN ENABLES THE CREATION OF CHAINS OF CALLS TO LLMS OR OTHER TOOLS, ALLOWING FOR MORE COMPLEX WORKFLOWS AND INTERACTIONS.\n\n3. **MEMORY**: THE FRAMEWORK CAN MAINTAIN CONTEXT OVER MULTIPLE INTERACTIONS, WHICH IS USEFUL FOR APPLICATIONS THAT REQUIRE A CONVERSATIONAL MEMORY.\n\n4. **INTEGRATION WITH EXTERNAL DATA SOURCES**: LANGCHAIN CAN CONNECT TO VARIOUS DATA SOURCES, ENABLING LLMS TO ACCESS AND UTILIZE EXTERNAL INFORMATION.\n\n5. **AGENTS**: IT SUPPORTS THE DEVELOPMENT OF AGENTS THAT CAN MAKE D

## Real-Time Streaming

In [17]:
from langchain.llms import OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = OpenAI(
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

print("Streaming response:\n")
result = llm("Tell me a story.")

print("\nFinal result:")
print(result)

Streaming response:



Once upon a time, in a faraway kingdom, there lived a young princess named Isabella. She was known for her beauty, kindness, and intelligence. Her parents, the king and queen, loved her dearly and wanted nothing but the best for her.

One day, a wicked sorcerer named Malakar attacked the kingdom with his army of dark creatures. They destroyed everything in their path and captured the king and queen. Isabella managed to escape and sought refuge in a nearby village.

The villagers welcomed her with open arms and promised to help her rescue her parents. Isabella was determined to save her kingdom and her parents, so she set out on a journey to find the legendary sword of light, which was said to have the power to defeat Malakar.

On her journey, Isabella encountered many challenges and obstacles, but she never gave up. She met a group of brave warriors who joined her on her quest. They traveled through treacherous forests, crossed raging rivers, and climbed steep mo